In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_img = nib.load(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\roughRR_images_mphys\1617496_plan.nii")
X = ct_img.get_fdata()
X = (X - np.min(X)) / (np.max(X) - np.min(X))
spacing = ct_img.header.get_zooms()

def show_slice(slice_index, plane):
    plt.figure(figsize=(6, 6))
    if plane == 'Axial':
        img_slice = np.rot90(X[:, :, slice_index])
        aspect = spacing[1] / spacing[0]
    elif plane == 'Coronal':
        img_slice = np.flipud(np.rot90(X[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]
    elif plane == 'Sagittal':
        img_slice = np.flipud(np.rot90(X[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]
    plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)
    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")
    clear_output(wait=True)
    display(plt.gcf())
    plt.close()

plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=X.shape[2] // 2,
    min=0,
    max=X.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = X.shape[2] - 1
        slice_slider.value = X.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = X.shape[1] - 1
        slice_slider.value = X.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = X.shape[0] - 1
        slice_slider.value = X.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(show_slice, slice_index=slice_slider, plane=plane_dropdown)


interactive(children=(IntSlider(value=256, description='Slice:', layout=Layout(width='80%'), max=511), Dropdow…

<function __main__.show_slice(slice_index, plane)>

In [ ]:
#separate sliders
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

ct_paths = [
    r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\roughRR_images_mphys\1720903_plan.nii",
    r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\roughRR_images_mphys\1720903_TP1.nii",
    r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\roughRR_images_mphys\1720903_TP2.nii",
    r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\roughRR_images_mphys\1720903_TP3.nii",
]

titles = ["Plan", "Timepoint 1", "Timepoint 2", "Timepoint 3"]

cts_raw = [nib.load(p) for p in ct_paths]
cts = [img.get_fdata() for img in cts_raw]
cts = [(X - np.min(X)) / (np.max(X) - np.min(X)) for X in cts]
spacings = [img.header.get_zooms() for img in cts_raw]

# Create a dedicated output area for the plot
plot_out = widgets.Output()

plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

sliders = []
for i, X in enumerate(cts):
    sliders.append(
        widgets.IntSlider(
            value=0, min=0, max=X.shape[1]-1, step=1,
            description=f"{titles[i]} slice",
            continuous_update=True, layout=widgets.Layout(width='95%')
        )
    )

def update_sliders(*args):
    plane = plane_dropdown.value
    for i, X in enumerate(cts):
        if plane == 'Axial':
            sliders[i].max = X.shape[1]-1
        elif plane == 'Coronal':
            sliders[i].max = X.shape[0]-1
        elif plane == 'Sagittal':
            sliders[i].max = X.shape[2]-1
        sliders[i].value = sliders[i].max // 2

plane_dropdown.observe(update_sliders, names='value')
update_sliders()

def plot_all(*args):
    plane = plane_dropdown.value

    with plot_out:
        clear_output(wait=True)
        plt.figure(figsize=(22, 5))

        for i, X in enumerate(cts):
            idx = sliders[i].value

            if plane == 'Axial':
                img_slice = np.flipud(X[:, idx, :])
                aspect = spacings[i][2] / spacings[i][0]
            elif plane == 'Coronal':
                img_slice = np.flipud(X[idx, :, :])
                aspect = spacings[i][1] / spacings[i][2]
            elif plane == 'Sagittal':
                img_slice = np.rot90(X[:, :, idx])
                aspect = spacings[i][1] / spacings[i][0]

            plt.subplot(1, 4, i+1)
            plt.imshow(img_slice, cmap='gray', origin='lower', aspect=aspect)
            plt.title(f"{titles[i]} – {plane} slice {idx}")
            plt.axis("off")

        plt.show()

for s in sliders:
    s.observe(plot_all, names='value')

plane_dropdown.observe(plot_all, names='value')

ui = widgets.VBox([plane_dropdown] + sliders)

display(ui)
display(plot_out)

plot_all()


Output()